In [70]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

In [71]:
!wget -O loan_train.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv


--2019-05-04 11:20:00--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.193
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23101 (23K) [text/csv]
Saving to: ‘loan_train.csv’

100%[======================================>] 23,101      --.-K/s   in 0.07s   

2019-05-04 11:20:00 (304 KB/s) - ‘loan_train.csv’ saved [23101/23101]



# LOAD CSV

In [72]:
df = pd.read_csv('loan_train.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,0,0,PAIDOFF,1000,30,9/8/2016,10/7/2016,45,High School or Below,male
1,2,2,PAIDOFF,1000,30,9/8/2016,10/7/2016,33,Bechalor,female
2,3,3,PAIDOFF,1000,15,9/8/2016,9/22/2016,27,college,male
3,4,4,PAIDOFF,1000,30,9/9/2016,10/8/2016,28,college,female
4,6,6,PAIDOFF,1000,30,9/9/2016,10/8/2016,29,college,male


In [73]:
df.shape

(346, 10)

In [74]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,0,0,PAIDOFF,1000,30,2016-09-08,2016-10-07,45,High School or Below,male
1,2,2,PAIDOFF,1000,30,2016-09-08,2016-10-07,33,Bechalor,female
2,3,3,PAIDOFF,1000,15,2016-09-08,2016-09-22,27,college,male
3,4,4,PAIDOFF,1000,30,2016-09-09,2016-10-08,28,college,female
4,6,6,PAIDOFF,1000,30,2016-09-09,2016-10-08,29,college,male


In [75]:
df['loan_status'].value_counts()

PAIDOFF       260
COLLECTION     86
Name: loan_status, dtype: int64

In [ ]:
# notice: installing seaborn might takes a few minutes
!conda install -c anaconda seaborn -y

Solving environment: - 

In [ ]:
import seaborn as sns

bins = np.linspace(df.Principal.min(), df.Principal.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set3", col_wrap=2)
g.map(plt.hist, 'Principal', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
bins = np.linspace(df.age.min(), df.age.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set2", col_wrap=2)
g.map(plt.hist, 'age', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
df['dayofweek'] = df['effective_date'].dt.dayofweek
bins = np.linspace(df.dayofweek.min(), df.dayofweek.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'dayofweek', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()

In [ ]:
df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df.head()

In [ ]:
df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)


In [ ]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

In [ ]:
df.groupby(['education'])['loan_status'].value_counts(normalize=True)

In [ ]:
df[['Principal','terms','age','Gender','education']].head()

In [ ]:
Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()

In [ ]:
X = Feature
X[0:5]

In [ ]:
y = df['loan_status'].values
y[0:5]

In [ ]:
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

# KNN

In [ ]:
from sklearn.model_selection import train_test_split
#classifier
from sklearn.neighbors import KNeighborsClassifier
from random import seed
seed(2019-5-4)
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=4)


In [ ]:
K_max = 21
scores = [];
for n in range(1, K_max):
    knn =  KNeighborsClassifier(n_neighbors = n)
    mod = knn.fit(X_train,y_train)
    knn_score= knn.score(X_test, y_test)
    scores.append(knn_score)

In [ ]:
scores = np.array(scores)
plt.plot(np.arange(1, K_max), scores)
k_best = np.argmax(scores)+1 # define the best k
val_best = np.max(scores)
print("The best  number of centroids is {0}, with the mean accuracy of {1:.4f}.".format(k_best, val_best))
knn_best = KNeighborsClassifier(n_neighbors = k_best)
knn_best = knn_best.fit(X_train,y_train)

In [ ]:
# Final model KNN
neigh = KNeighborsClassifier(n_neighbors = 2).fit(X_train,y_train)

# DECISION TREE

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree_K_max = 15
tree_scores = [];
for n in range(1, tree_K_max):
    d_tree = DecisionTreeClassifier( criterion="entropy", max_depth=n )
    mod = d_tree.fit(X_train,y_train)
    tree_score= d_tree.score(X_test, y_test)
    tree_scores.append(tree_score)

In [ ]:
tree_scores = np.array(tree_scores)
plt.plot(np.arange(1, tree_K_max), tree_scores)
tree_best = np.argmax(tree_scores)+1 # define the best k
tree_val_best = np.max(tree_scores)
print("The  number is {0}, with the mean accuracy of {1:.4f}.".format(tree_best, tree_val_best))
d_tree_best = DecisionTreeClassifier( criterion="entropy", max_depth= tree_best )
d_tree_best = d_tree_best.fit(X_train,y_train)

In [ ]:
y_train


# SVM

In [ ]:
from sklearn import svm
svm_K_max = 10
svm_scores = []

for n in range(1, svm_K_max):
    s_svm = svm.SVC(kernel='poly', degree=n)
    mod = s_svm.fit(X_train, y_train)
    svm_score= s_svm.score(X_test, y_test)
    svm_scores.append(svm_score)

In [ ]:
svm_scores = np.array(svm_scores)
plt.plot(np.arange(1, svm_K_max), svm_scores)
svm_best = np.argmax(svm_scores)+1 # define the best k
tsvm_val_best = np.max(svm_scores)
print("The  number is {0}, with the mean accuracy of {1:.4f}.".format(svm_best, tsvm_val_best))
s_svm_best = svm.SVC(kernel='poly', degree=svm_best )
s_svm_best = s_svm_best.fit(X_train,y_train)


## LR

In [ ]:
#This one, no K:
from sklearn.linear_model import LogisticRegression
l_r=LogisticRegression( solver='lbfgs')
l_r = l_r.fit(X_train, y_train)
l_r_score = l_r.score(X_test,  y_test)
print("The  best mean accuracy is {0:.4f}.".format(l_r_score))

In [ ]:
#MODEL EVALUATION 
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

In [ ]:
!wget -O loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv

In [ ]:
# Just following  procedure of the preprocessing data
test_df = pd.read_csv('loan_test.csv')

test_df['due_date'] = pd.to_datetime(test_df['due_date'])
test_df['effective_date'] = pd.to_datetime(test_df['effective_date'])

test_df['dayofweek'] = test_df['effective_date'].dt.dayofweek
test_df['weekend'] = test_df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
test_df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)

test_Feature = test_df[['Principal','terms','age','Gender','weekend']]
test_Feature = pd.concat([test_Feature,pd.get_dummies(test_df['education'])], axis=1)
test_Feature.drop(['Master or Above'], axis = 1,inplace=True)
test_Feature.head()

In [ ]:
x_test_f = test_Feature
x_test_f = preprocessing.StandardScaler().fit(x_test_f.astype(float)).transform(x_test_f.astype(float)) 
y_test_f = test_df['loan_status'].value_counts()

In [ ]:
### KNN
yhat_knn= knn_best.predict(x_test_f)
jac_knn = jaccard_similarity_score(yhat_knn, y_test_f)
f1_knn = f1_score(yhat_knn, y_test_f, average='weighted')
print("KNN: {0:.3f}, {1:.3f}".format(jac_knn, f1_knn))
### Tree
y_hat_tree = d_tree_best.predict(x_test_f)
jac_tree = jaccard_similarity_score(y_hat_tree, y_test_f)
f1_tree = f1_score(y_hat_tree, y_test_f, average='weighted')
print("Decision tree: {0:.3f}, {1:.3f}".format(jac_tree, f1_tree))
### SVM
y_hat_svm = s_svm_best.predict(x_test_f)
jac_svm = jaccard_similarity_score(y_hat_svm, y_test_f)
f1_svm = f1_score(y_hat_svm, y_test_f, average='weighted')
print("Decision tree: {0:.3f}, {1:.3f}".format(jac_svm, f1_svm))

### Logi Res
y_hat_lr = l_r.predict(x_test_f)
jac_lr = jaccard_similarity_score(y_hat_lr, y_test_f)
f1_lr = f1_score(y_hat_lr, y_test_f, average='weighted')
###
y_hat_lr[y_hat_lr == "PAIDOFF"] =1 
y_hat_lr[y_hat_lr == "COLLECTION"] = 0
y_test_f[y_test_f == "PAIDOFF"] =1 
y_test_f[y_test_f == "COLLECTION"] = 0
y_hat_lr = y_hat_lr.astype(float)
y_test_f = y_test_f.astype(float)

log_loss_lr = log_loss(y_hat_lr, y_test_f, normalize=True)

print("Decision tree: {0:.3f}, {1:.3f}, {2:.3f}".format(jac_lr, f1_lr, log_loss_lr))

# Result
KNN: 0.667, 0.718
Decision tree: 0.722, 0.774
Decision tree: 0.759, 0.823
Decision tree: 0.759, 0.847, 8.315